In [1]:
import pandas as pd

# 1. 데이터 로드
data = pd.read_csv('spy_2020_2022_with_rq.csv')

# 2. 컬럼명 정리
data.columns = data.columns.str.strip().str.replace('[', '').str.replace(']', '')

# 3. 필요한 컬럼만 확인
needed_cols = [
    'UNDERLYING_LAST', 'STRIKE', 'DTE',
    'C_IV', 'P_IV',
    'r', 'q',
    'C_LAST', 'P_LAST'  # 나중에 market price와 비교할 때 사용
]
assert all(col in data.columns for col in needed_cols), "필수 컬럼이 없습니다."

# 4. T 계산
data['T'] = data['DTE'] / 365.0

# 5. 결과 확인
print(data[['UNDERLYING_LAST', 'STRIKE', 'T', 'C_IV', 'P_IV', 'r', 'q']].head())


C:\Users\young\AppData\Local\Temp\ipykernel_16276\936928799.py:4: DtypeWarning: Columns (8,9,10,11,12,13,15,17,18,20,21,23,24,25,26,27,28,29) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('spy_2020_2022_with_rq.csv')


   UNDERLYING_LAST  STRIKE    T C_IV       P_IV     r     q
0           451.85   270.0  0.0        3.412490  0.07  1.28
1           451.85   275.0  0.0        3.297370  0.07  1.28
2           451.85   280.0  0.0        3.183300  0.07  1.28
3           451.85   285.0  0.0        3.072170  0.07  1.28
4           451.85   290.0  0.0        2.962300  0.07  1.28


In [16]:
BS_data = data[['UNDERLYING_LAST', 'STRIKE', 'T', 'C_IV', 'P_IV', 'r', 'q']]

In [17]:
import numpy as np
import scipy.stats as st

BS_data = BS_data.replace(' ', np.nan)

BS_data = BS_data.astype({
    'UNDERLYING_LAST': float,
    'STRIKE': float,
    'T': float,
    'C_IV': float,
    'P_IV': float,
    'r': float,
    'q': float
})

# 1. 필요한 값 추출
S = BS_data['UNDERLYING_LAST'].values  # 기초자산 가격
K = BS_data['STRIKE'].values            # 행사가
T = BS_data['T'].values                 # 잔존 만기
r = BS_data['r'].values                 # 무위험 이자율
q = BS_data['q'].values                 # 배당수익률
sigma_call = BS_data['C_IV'].values      # 콜 IV
sigma_put = BS_data['P_IV'].values       # 풋 IV

# 2. d1, d2 계산 함수
def calc_d1_d2(S, K, T, r, q, sigma):
    d1 = (np.log(S / K) + (r - q + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return d1, d2

# 3. BSM 계산 함수
def calc_bsm_prices_and_greeks(S, K, T, r, q, sigma, option_type='call'):
    d1, d2 = calc_d1_d2(S, K, T, r, q, sigma)

    if option_type == 'call':
        price = np.exp(-q * T) * S * st.norm.cdf(d1) - np.exp(-r * T) * K * st.norm.cdf(d2)
        delta = np.exp(-q * T) * st.norm.cdf(d1)
    elif option_type == 'put':
        price = np.exp(-r * T) * K * st.norm.cdf(-d2) - np.exp(-q * T) * S * st.norm.cdf(-d1)
        delta = -np.exp(-q * T) * st.norm.cdf(-d1)

    gamma = np.exp(-q * T) * st.norm.pdf(d1) / (S * sigma * np.sqrt(T))
    vega = S * np.exp(-q * T) * st.norm.pdf(d1) * np.sqrt(T)
    theta = (- (S * sigma * np.exp(-q * T) * st.norm.pdf(d1)) / (2 * np.sqrt(T)) 
             - r * K * np.exp(-r * T) * (st.norm.cdf(d2) if option_type == 'call' else st.norm.cdf(-d2))
             + q * S * np.exp(-q * T) * (st.norm.cdf(d1) if option_type == 'call' else st.norm.cdf(-d1))
            )
    rho = (K * T * np.exp(-r * T) * (st.norm.cdf(d2) if option_type == 'call' else -st.norm.cdf(-d2)))

    return price, delta, gamma, vega, theta, rho

# 4. 콜옵션 계산
BSM_C_Price, BSM_C_Delta, BSM_C_Gamma, BSM_C_Vega, BSM_C_Theta, BSM_C_Rho = calc_bsm_prices_and_greeks(
    S, K, T, r, q, sigma_call, option_type='call'
)

# 5. 풋옵션 계산
BSM_P_Price, BSM_P_Delta, BSM_P_Gamma, BSM_P_Vega, BSM_P_Theta, BSM_P_Rho = calc_bsm_prices_and_greeks(
    S, K, T, r, q, sigma_put, option_type='put'
)

# 6. 결과 저장
BS_data['BSM_C_Price'] = BSM_C_Price
BS_data['BSM_C_Delta'] = BSM_C_Delta
BS_data['BSM_C_Gamma'] = BSM_C_Gamma
BS_data['BSM_C_Vega']  = BSM_C_Vega
BS_data['BSM_C_Theta'] = BSM_C_Theta
BS_data['BSM_C_Rho']   = BSM_C_Rho

BS_data['BSM_P_Price'] = BSM_P_Price
BS_data['BSM_P_Delta'] = BSM_P_Delta
BS_data['BSM_P_Gamma'] = BSM_P_Gamma
BS_data['BSM_P_Vega']  = BSM_P_Vega
BS_data['BSM_P_Theta'] = BSM_P_Theta
BS_data['BSM_P_Rho']   = BSM_P_Rho

# 7. 최종 결과 확인
print(BS_data.head())

C:\Users\young\AppData\Local\Temp\ipykernel_16276\616917045.py:27: RuntimeWarning: divide by zero encountered in divide
  d1 = (np.log(S / K) + (r - q + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
C:\Users\young\AppData\Local\Temp\ipykernel_16276\616917045.py:27: RuntimeWarning: invalid value encountered in divide
  d1 = (np.log(S / K) + (r - q + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
C:\Users\young\AppData\Local\Temp\ipykernel_16276\616917045.py:42: RuntimeWarning: invalid value encountered in divide
  gamma = np.exp(-q * T) * st.norm.pdf(d1) / (S * sigma * np.sqrt(T))
C:\Users\young\AppData\Local\Temp\ipykernel_16276\616917045.py:44: RuntimeWarning: invalid value encountered in divide
  theta = (- (S * sigma * np.exp(-q * T) * st.norm.pdf(d1)) / (2 * np.sqrt(T))


   UNDERLYING_LAST  STRIKE    T  C_IV     P_IV     r     q  BSM_C_Price  \
0           451.85   270.0  0.0   NaN  3.41249  0.07  1.28          NaN   
1           451.85   275.0  0.0   NaN  3.29737  0.07  1.28          NaN   
2           451.85   280.0  0.0   NaN  3.18330  0.07  1.28          NaN   
3           451.85   285.0  0.0   NaN  3.07217  0.07  1.28          NaN   
4           451.85   290.0  0.0   NaN  2.96230  0.07  1.28          NaN   

   BSM_C_Delta  BSM_C_Gamma  BSM_C_Vega  BSM_C_Theta  BSM_C_Rho  BSM_P_Price  \
0          NaN          NaN         NaN          NaN        NaN          0.0   
1          NaN          NaN         NaN          NaN        NaN          0.0   
2          NaN          NaN         NaN          NaN        NaN          0.0   
3          NaN          NaN         NaN          NaN        NaN          0.0   
4          NaN          NaN         NaN          NaN        NaN          0.0   

   BSM_P_Delta  BSM_P_Gamma  BSM_P_Vega  BSM_P_Theta  BSM_P_Rho  
0 

In [18]:
BS_data

,UNDERLYING_LAST,STRIKE,T,C_IV,P_IV,r,q,BSM_C_Price,BSM_C_Delta,BSM_C_Gamma,BSM_C_Vega,BSM_C_Theta,BSM_C_Rho,BSM_P_Price,BSM_P_Delta,BSM_P_Gamma,BSM_P_Vega,BSM_P_Theta,BSM_P_Rho
0,451.85,270.0,0.0,NaN,3.41249,0.07,1.28,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,-0.000000,NaN,0.0,NaN,-0.000000
1,451.85,275.0,0.0,NaN,3.29737,0.07,1.28,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,-0.000000,NaN,0.0,NaN,-0.000000
2,451.85,280.0,0.0,NaN,3.18330,0.07,1.28,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,-0.000000,NaN,0.0,NaN,-0.000000
3,451.85,285.0,0.0,NaN,3.07217,0.07,1.28,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,-0.000000,NaN,0.0,NaN,-0.000000
4,451.85,290.0,0.0,NaN,2.96230,0.07,1.28,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,-0.000000,NaN,0.0,NaN,-0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3589074,380.36,570.0,2.8,0.18659,0.00034,0.08,1.28,1.558427e-33,1.624047e-34,1.638824e-35,1.238707e-30,3.297809e-32,1.685987e-31,445.049152,-0.027764,0.0,0.0,-22.931363,-1275.706954
3589075,380.36,575.0,2.8,0.18477,0.00000,0.08,1.28,2.544566e-34,2.708899e-35,2.794459e-36,2.091592e-31,5.483504e-33,2.813751e-32,449.045728,-0.027764,NaN,0.0,-23.251089,-1286.897366
3589076,380.36,580.0,2.8,0.18793,0.00041,0.08,1.28,2.301619e-33,2.375820e-34,2.374073e-35,1.807332e-30,4.797203e-32,2.465822e-31,453.042304,-0.027764,0.0,0.0,-23.570815,-1298.087778
3589077,380.36,585.0,2.8,0.18419,-0.00037,0.08,1.28,7.931234e-35,8.532752e-36,8.898566e-37,6.639475e-32,1.717171e-33,8.865374e-33,0.000000,-0.000000,-0.0,0.0,0.000000,-0.000000
